In [1]:
import random, time, turtle, math
import numpy as np
from deap import base, creator, tools
from IPython import display as dis
import matplotlib.pyplot as plt

In [2]:
class DisplayGame:
    def __init__(self, XSIZE, YSIZE):
        # SCREEN
        self.win = turtle.Screen()
        self.win.title("EVCO Snake game")
        self.win.bgcolor("grey")
        self.win.setup(width=(XSIZE*20)+40,height=(YSIZE*20)+40)
        #self.win.screensize((XSIZE*20)+20,(YSIZE*20)+20)
        self.win.tracer(0)

        #Snake Head
        self.head = turtle.Turtle()
        self.head.shape("square")
        self.head.color("black")

        # Snake food
        self.food = turtle.Turtle()
        self.food.shape("circle")
        self.food.color("yellow")
        self.food.penup()
        self.food.shapesize(0.55, 0.55)
        self.segments = []

    def reset(self, snake):
        for i in self.segments:
            i.reset
        self.segments = []
        self.head.penup()
        self.food.goto(-500, -500)
        self.head.goto(-500, -500)
        for i in range(len(snake)-1):
            self.add_snake_segment()
        self.update_segment_positions(snake)
       
    def update_food(self,new_food):
        self.food.goto(((new_food[1]-9)*20)+20, (((9-new_food[0])*20)-10)-20)
        
    def update_segment_positions(self, snake):
        self.head.goto(((snake[0][1]-9)*20)+20, (((9-snake[0][0])*20)-10)-20)
        for i in range(len(self.segments)):
            self.segments[i].goto(((snake[i+1][1]-9)*20)+20, (((9-snake[i+1][0])*20)-10)-20)

    def add_snake_segment(self):
        self.new_segment = turtle.Turtle()
        self.new_segment.speed(0)
        self.new_segment.shape("square")
        self.new_segment.color(random.choice(["green",'black','red','blue']))
        self.new_segment.penup()
        self.segments.append(self.new_segment)

In [3]:
class snake:
    def __init__(self, _XSIZE, _YSIZE):
        self.XSIZE = _XSIZE
        self.YSIZE = _YSIZE
        self.reset()

    def reset(self):
        self.snake = [[8,10], [8,9], [8,8], [8,7], [8,6], [8,5], [8,4], [8,3], [8,2], [8,1],[8,0] ]
        self.food = self.place_food()
        self.ahead = []
        self.snake_direction = "right"

    def place_food(self):
        self.food = [random.randint(1, (YSIZE-2)), random.randint(1, (XSIZE-2))]
        while (self.food in self.snake):
            self.food = [random.randint(1, (YSIZE-2)), random.randint(1, (XSIZE-2))]
        return( self.food )
    
    def update_snake_position(self):
        self.snake.insert(0, [self.snake[0][0] + (self.snake_direction == "down" and 1) + (self.snake_direction == "up" and -1), self.snake[0][1] + (self.snake_direction == "left" and -1) + (self.snake_direction == "right" and 1)])

    def food_eaten(self):
        if self.snake[0] == self.food:                                            
            return True
        else:    
            last = self.snake.pop()  
            return False
            
    def snake_turns_into_self(self):
        if self.snake[0] in self.snake[1:]:
            return True
        else:
            return False

    def snake_hit_wall(self):
        if self.snake[0][0] == 0 or self.snake[0][0] == (YSIZE-1) or self.snake[0][1] == 0 or self.snake[0][1] == (XSIZE-1):
            return True
        else:
            return False

        #sensing functions 
    def getAheadLocation(self):
        self.ahead = [ self.snake[0][0] + (self.snake_direction == "down" and 1) + (self.snake_direction == "up" and -1), self.snake[0][1] + (self.snake_direction == "left" and -1) + (self.snake_direction == "right" and 1)] 
    
    def sense_wall_ahead(self):
        self.getAheadLocation()
        return( self.ahead[0] == 0 or self.ahead[0] == (YSIZE-1) or self.ahead[1] == 0 or self.ahead[1] == (XSIZE-1) )
        
    def sense_food_ahead(self):
        self.getAheadLocation()
        return self.food == self.ahead

    def sense_tail_ahead(self):
        self.getAheadLocation()
        return self.ahead in self.snake
    
    def snake_head_posX(self):
        return self.snake[0][0]
    
    def snake_head_posY(self):
        return self.snake[0][1]
    
    def food_posX(self):
        return self.food[0]
    
    def food_posY(self):
        return self.food[1]
    
    def food_relativeX(self):
        return self.snake[0][0] - self.food[0]
    
    def food_relativeY(self):
        return self.snake[0][1] - self.food[1]
    
    def getState(self):
        current_direction = snake_game.snake_direction
       
        direction = ["left", "up", "down", "right"]
        danger_location = [0,0,0,0]
        current_direction_facing = [0,0,0,0]
        food_direction_bool=[0,0,0,0]
        
        
        count=0
        for i in direction:
            snake_game.snake_direction = i
            if snake_game.sense_tail_ahead():
                danger_location[count] = True
            elif snake_game.sense_wall_ahead():
                danger_location[count] = True
            else:
                danger_location[count] = False
            count+=1 
        
        dir_l = self.snake_direction == "left"
        dir_r = self.snake_direction == "right"
        dir_u = self.snake_direction == "up"
        dir_d = self.snake_direction == "down"
        
        state = [
            
            # Move direction
            danger_location[0],
            danger_location[1],
            danger_location[2],
            danger_location[3],
            
            dir_l,
            dir_r,
            dir_u,
            dir_d,

            # Food location 
            #Checks to see where the food is in relation to the position of the snakes head
            self.food[0] > self.snake[0][0],  # food left
            self.food[0] < self.snake[0][0],  # food right
            self.food[1] > self.snake[0][1],  # food up
            self.food[1] < self.snake[0][1]  # food down

            
        ]
        
        return state
    
    

In [4]:
class NN(object):
    def __init__(self, numInput, numHidden1,numOutput):
        self.fitness = 0,
        self.numInput = numInput + 1 # Add bias node from input to hidden layer 1 only,
        self.numHidden1 = numHidden1 # Feel free to adapt the code to add more biases if you wish,
        self.numOutput = numOutput

        self.w_i_h1 = np.random.randn(self.numHidden1, self.numInput) 
        self.w_h1_o = np.random.randn(self.numOutput, self.numHidden1)

        self.ReLU = lambda x : max(0,x)

    def sigmoid(self,x):
        try:
            ans = (1 / (1 + math.exp(-x)))
        except OverflowError:
            ans = float('inf')
        return ans

In [5]:
class NN(NN):
        def feedForward(self, inputs):
            inputsBias = inputs[:]
            inputsBias.insert(len(inputs),1)             # Add bias input
    
            h1 = np.dot(self.w_i_h1, inputsBias)         # feed input to hidden layer 1
            h1 = [self.ReLU(x) for x in h1]              # Activate hidden layer1
              
            output = np.dot(self.w_h1_o, h1)             # feed to output layer
            output = [self.sigmoid(x) for x in output]   # Activate output layer
            return output

In [6]:
class NN(NN):
        
        def getWeightsLinear(self):
            flat_w_i_h1 = list(self.w_i_h1.flatten())
            flat_w_h2_o = list(self.w_h1_o.flatten())
            return( flat_w_i_h1 + flat_w_h1_o )
    
        def setWeightsLinear(self, Wgenome):
            numWeights_I_H1 = self.numHidden1 * self.numInput
            numWeights_H2_O = self.numOutput * self.numHidden1
    
            self.w_i_h1 = np.array(Wgenome[:numWeights_I_H1])
            self.w_i_h1 = self.w_i_h1.reshape((self.numHidden1, self.numInput))
            
    
            self.w_h1o = np.array(Wgenome[(numWeights_I_H1):])
            self.w_h1o = self.w_h1o.reshape((self.numOutput, self.numHidden1))

In [7]:
XSIZE = YSIZE = 16 # Number of grid cells in each direction (do not change this)
HEADLESS = True

snake_game = snake(XSIZE,YSIZE)

if HEADLESS == False:
    display = DisplayGame(XSIZE,YSIZE)

tournamentSize = 20
muProb = 0.1
population = 100
generations = 500
muSigma = 0.2


inputLayer = 12
HiddenNodes = 32
outputLayer = 4

IND_SIZE = ((inputLayer+1) * (HiddenNodes)) + ((HiddenNodes) * (outputLayer))

myNet = NN(inputLayer, HiddenNodes, outputLayer)

In [8]:
HEADLESS = True

In [9]:
def run_game(AIbrain, display,game, headless):
    
    if headless == False:
        display = DisplayGame(XSIZE,YSIZE)
    score = 0
    reward_score=0
    total_moves=0
    moves = 0
    time_max = 60
    game.reset()
    if not headless:
        display.reset(game.snake)
        display.win.update()
    game.place_food()
    game_over = False
    snake_direction = "right"

    flag = True
    while not game_over:
        

        state = game.getState()          
        output = AIbrain.feedForward(state)
        new_direction = np.argmax(output, axis=0)
        
        if new_direction == 0:
            snake_direction ="left"
        elif new_direction == 1:
            snake_direction ="up"
        elif new_direction == 2:
            snake_direction ="down"
        elif new_direction == 3:
            snake_direction ="right"
        game.snake_direction = snake_direction       
        game.update_snake_position()
        
    
        if game.food_eaten():
            moves = 0
            score += 1
            game.place_food()   
            if not headless: display.add_snake_segment()

        if game.snake_turns_into_self() or game.snake_hit_wall():
            game_over = True
            
        if not headless:       
            display.update_food(game.food)
            display.update_segment_positions(game.snake)
            display.win.update()
            time.sleep(0.05) # Change this to modify the speed the game runs at when displayed.
            
        if moves > time_max:
            game_over=True
        moves += 1
        total_moves+= 1
        
    if not headless: 
        turtle.clearscreen() 
        
    scorelist.append(score)#print(score)
    
    print("Score: "+str(score))
    return (score)

In [10]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, -1.0, 1.0)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)



In [11]:
scorelist=[]
meanlist=[]
total_score =0
def evaluate(indiv, myNet,g):
    fitness = 0
    myNet.setWeightsLinear(indiv)
    headless = True
    if g > 498:
        headless = False
    fitness = run_game(myNet, display, snake_game, headless)
    return fitness,



In [12]:
toolbox.register("evaluate", evaluate)
toolbox.register("select", tools.selTournament, tournsize=tournamentSize)

toolbox.register("mutate", tools.mutGaussian, mu=0.0, sigma=muSigma, indpb=muProb)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)



In [13]:
stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [14]:
logbook = tools.Logbook()

pop = toolbox.population(n=50)

In [15]:
fitnesses = [toolbox.evaluate(indiv, myNet,0) for indiv in pop]
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 3
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0


In [16]:
for g in range(generations):
    print("-- Generation %i --" % g)

    offspring = toolbox.select(pop, len(pop))
    offspring = list(map(toolbox.clone, offspring))

    for mutant in offspring:
        toolbox.mutate(mutant)
        del mutant.fitness.values

    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = [toolbox.evaluate(indiv, myNet,g) for indiv in invalid_ind]
    for ind, fit in zip(invalid_ind, fitnesses):
         ind.fitness.values = fit
    pop[:] = offspring
    record = stats.compile(pop)
    logbook.record(gen=g, **record)

-- Generation 0 --
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 1
Score: 1
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
-- Generation 1 --
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 1
Score: 1
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
-- Generation 2 --
Score: 0
Score: 0
Score: 1
Score: 0
Score: 

Score: 4
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 2
Score: 0
Score: 0
Score: 2
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 1
Score: 1
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 2
Score: 0
Score: 0
Score: 2
-- Generation 21 --
Score: 1
Score: 0
Score: 1
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 3
Score: 1
Score: 2
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 1
Score: 1
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 1
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 0
Score: 1
-- Generation 22 --
Score: 0
Score: 1
Score: 0
Score: 1
Score: 0
Score: 1
Score: 0
Score: 0
Score:

Score: 1
Score: 1
Score: 0
Score: 0
Score: 1
Score: 0
Score: 1
Score: 1
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 1
Score: 0
Score: 1
Score: 1
Score: 0
Score: 2
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 0
Score: 1
Score: 2
Score: 2
Score: 0
Score: 1
-- Generation 39 --
Score: 3
Score: 2
Score: 0
Score: 1
Score: 0
Score: 1
Score: 0
Score: 0
Score: 1
Score: 1
Score: 0
Score: 1
Score: 2
Score: 0
Score: 0
Score: 2
Score: 2
Score: 1
Score: 1
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 1
Score: 0
Score: 0
Score: 1
Score: 3
Score: 1
Score: 2
Score: 0
Score: 0
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 2
Score: 0
Score: 2
Score: 1
Score: 0
Score: 0
Score: 3
Score: 1
Score: 0
Score: 1
Score: 0
Score: 0
-- Generation 40 --
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 0
Score: 0
Score: 1
Score: 0
Score: 0
Score: 1
Score: 5
Score: 0
Score: 0
Score: 0
Score: 1
Score:

Score: 6
Score: 7
Score: 2
Score: 8
Score: 10
Score: 7
Score: 3
Score: 3
Score: 12
Score: 1
Score: 2
Score: 2
Score: 9
Score: 0
Score: 16
Score: 10
Score: 2
Score: 9
Score: 5
Score: 8
Score: 6
Score: 9
Score: 6
Score: 5
Score: 0
Score: 6
Score: 4
Score: 0
Score: 2
Score: 1
Score: 5
Score: 1
Score: 1
Score: 3
Score: 7
Score: 0
Score: 5
Score: 3
Score: 5
Score: 3
Score: 4
Score: 1
Score: 24
Score: 4
Score: 1
Score: 4
Score: 1
Score: 15
-- Generation 58 --
Score: 3
Score: 5
Score: 3
Score: 10
Score: 1
Score: 6
Score: 8
Score: 2
Score: 0
Score: 2
Score: 1
Score: 3
Score: 1
Score: 5
Score: 4
Score: 8
Score: 1
Score: 2
Score: 3
Score: 2
Score: 1
Score: 17
Score: 4
Score: 3
Score: 6
Score: 1
Score: 4
Score: 0
Score: 12
Score: 1
Score: 0
Score: 4
Score: 9
Score: 13
Score: 4
Score: 4
Score: 2
Score: 1
Score: 3
Score: 4
Score: 3
Score: 4
Score: 3
Score: 5
Score: 0
Score: 0
Score: 1
Score: 4
Score: 8
Score: 6
-- Generation 59 --
Score: 1
Score: 2
Score: 4
Score: 0
Score: 6
Score: 6
Score: 7
Score

Score: 9
Score: 7
Score: 1
Score: 9
Score: 20
Score: 0
Score: 2
Score: 3
Score: 18
Score: 15
Score: 2
Score: 13
Score: 12
Score: 3
Score: 3
Score: 2
Score: 5
Score: 1
Score: 8
Score: 1
Score: 3
Score: 0
Score: 1
Score: 5
Score: 10
Score: 13
Score: 5
Score: 9
Score: 0
Score: 5
Score: 3
Score: 4
Score: 7
Score: 0
Score: 12
-- Generation 76 --
Score: 2
Score: 14
Score: 1
Score: 3
Score: 12
Score: 13
Score: 4
Score: 11
Score: 1
Score: 13
Score: 9
Score: 4
Score: 1
Score: 7
Score: 5
Score: 0
Score: 6
Score: 2
Score: 4
Score: 0
Score: 0
Score: 4
Score: 11
Score: 1
Score: 3
Score: 6
Score: 11
Score: 3
Score: 3
Score: 6
Score: 1
Score: 5
Score: 8
Score: 2
Score: 4
Score: 5
Score: 5
Score: 11
Score: 1
Score: 2
Score: 4
Score: 2
Score: 4
Score: 14
Score: 7
Score: 11
Score: 8
Score: 6
Score: 18
Score: 1
-- Generation 77 --
Score: 13
Score: 16
Score: 1
Score: 2
Score: 9
Score: 4
Score: 14
Score: 3
Score: 7
Score: 11
Score: 9
Score: 0
Score: 19
Score: 4
Score: 5
Score: 4
Score: 3
Score: 3
Score: 9


Score: 25
Score: 17
Score: 3
-- Generation 93 --
Score: 14
Score: 3
Score: 2
Score: 8
Score: 4
Score: 14
Score: 14
Score: 3
Score: 16
Score: 8
Score: 14
Score: 11
Score: 23
Score: 1
Score: 21
Score: 3
Score: 9
Score: 1
Score: 22
Score: 6
Score: 10
Score: 1
Score: 9
Score: 4
Score: 17
Score: 24
Score: 11
Score: 22
Score: 12
Score: 14
Score: 2
Score: 5
Score: 9
Score: 29
Score: 5
Score: 18
Score: 13
Score: 12
Score: 4
Score: 28
Score: 3
Score: 0
Score: 1
Score: 23
Score: 10
Score: 17
Score: 5
Score: 11
Score: 4
Score: 7
-- Generation 94 --
Score: 13
Score: 1
Score: 9
Score: 3
Score: 4
Score: 10
Score: 13
Score: 5
Score: 15
Score: 4
Score: 12
Score: 10
Score: 2
Score: 1
Score: 13
Score: 11
Score: 3
Score: 12
Score: 7
Score: 16
Score: 10
Score: 3
Score: 12
Score: 8
Score: 17
Score: 23
Score: 7
Score: 1
Score: 4
Score: 1
Score: 14
Score: 16
Score: 8
Score: 1
Score: 17
Score: 6
Score: 9
Score: 5
Score: 7
Score: 2
Score: 6
Score: 9
Score: 15
Score: 4
Score: 19
Score: 11
Score: 6
Score: 6
Scor

Score: 7
Score: 0
Score: 7
Score: 0
Score: 0
Score: 10
Score: 0
Score: 7
Score: 2
Score: 12
Score: 0
Score: 15
Score: 0
Score: 10
Score: 1
Score: 8
Score: 3
Score: 25
Score: 0
Score: 2
-- Generation 111 --
Score: 8
Score: 7
Score: 17
Score: 18
Score: 4
Score: 1
Score: 7
Score: 16
Score: 17
Score: 11
Score: 12
Score: 6
Score: 3
Score: 11
Score: 4
Score: 22
Score: 38
Score: 16
Score: 13
Score: 33
Score: 12
Score: 3
Score: 13
Score: 13
Score: 4
Score: 3
Score: 0
Score: 23
Score: 7
Score: 0
Score: 1
Score: 5
Score: 1
Score: 0
Score: 0
Score: 15
Score: 7
Score: 10
Score: 7
Score: 14
Score: 4
Score: 10
Score: 9
Score: 0
Score: 7
Score: 10
Score: 16
Score: 15
Score: 1
Score: 3
-- Generation 112 --
Score: 3
Score: 3
Score: 1
Score: 20
Score: 4
Score: 0
Score: 14
Score: 0
Score: 2
Score: 5
Score: 0
Score: 12
Score: 31
Score: 1
Score: 13
Score: 16
Score: 10
Score: 13
Score: 22
Score: 13
Score: 1
Score: 0
Score: 0
Score: 1
Score: 21
Score: 0
Score: 10
Score: 1
Score: 0
Score: 8
Score: 9
Score: 1


KeyboardInterrupt: 

In [ ]:
plt.ion()

counter = 0
for i in scorelist:
    counter+=1
    total_score+= i
    meanlist.append(total_score/counter)

        
    
def plot(scores, mean_scores):
    dis.clear_output(wait=True)
    dis.display(plt.gcf())
    plt.clf()
    plt.title('Training...')
    plt.xlabel('Number of Games')
    plt.ylabel('Score')
    plt.plot(scores)
    plt.plot(mean_scores)
    plt.ylim(ymin=0)
    plt.text(len(scores)-1, scores[-1], str(scores[-1]))
    plt.text(len(mean_scores)-1, mean_scores[-1], str(mean_scores[-1]))
    plt.show(block=False)
    plt.pause(.1)

In [ ]:
plot(scorelist,meanlist)


In [ ]:
import pickle
cp = dict(population=pop, generation=g, logbook=logbook, rndstate=random.getstate())
with open("snake2.pkl", "wb") as cp_file:
    pickle.dump(cp, cp_file)

In [ ]:
%matplotlib inline

gen = logbook.select("gen")
_min = logbook.select("min")
_max = logbook.select("max")
avgs = logbook.select("avg")
stds = logbook.select("std")

plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14) 
plt.rc('legend', fontsize=14)

fig, ax1 = plt.subplots()
line1 = ax1.plot(gen, avgs)
ax1.set_xlabel("Generation")
ax1.set_ylabel("Fitness")



print(max(scorelist))

In [ ]:
import pickle

with open("Snake.pkl", "rb") as savePoint_file:
    savePoint = pickle.load(savePoint_file)

pop = savePoint["population"]
start_gen = savePoint["generation"]
logbook = savePoint["logbook"]
random.setstate(savePoint["rndstate"])

with open("Snake1.pkl", "rb") as savePoint_file:
    savePoint = pickle.load(savePoint_file)

pop1 = savePoint["population"]
start_gen1 = savePoint["generation"]
logbook1 = savePoint["logbook"]
random.setstate(savePoint["rndstate"])

with open("Snake2.pkl", "rb") as savePoint_file:
    savePoint = pickle.load(savePoint_file)

pop2 = savePoint["population"]
start_gen2 = savePoint["generation"]
logbook2 = savePoint["logbook"]
random.setstate(savePoint["rndstate"])

with open("Snake3.pkl", "rb") as savePoint_file:
    savePoint = pickle.load(savePoint_file)

pop3 = savePoint["population"]
start_gen3 = savePoint["generation"]
logbook3 = savePoint["logbook"]
random.setstate(savePoint["rndstate"])

with open("Snake4.pkl", "rb") as savePoint_file:
    savePoint = pickle.load(savePoint_file)

pop4 = savePoint["population"]
start_gen4 = savePoint["generation"]
logbook4 = savePoint["logbook"]
random.setstate(savePoint["rndstate"])

with open("Snake5.pkl", "rb") as savePoint_file:
    savePoint = pickle.load(savePoint_file)

pop = savePoint["population"]
start_gen = savePoint["generation"]
logbook5 = savePoint["logbook"]
random.setstate(savePoint["rndstate"])

with open("Snake6.pkl", "rb") as savePoint_file:
    savePoint = pickle.load(savePoint_file)

pop1 = savePoint["population"]
start_gen1 = savePoint["generation"]
logbook6 = savePoint["logbook"]
random.setstate(savePoint["rndstate"])

with open("Snake7.pkl", "rb") as savePoint_file:
    savePoint = pickle.load(savePoint_file)

pop2 = savePoint["population"]
start_gen2 = savePoint["generation"]
logbook7 = savePoint["logbook"]
random.setstate(savePoint["rndstate"])

with open("Snake8.pkl", "rb") as savePoint_file:
    savePoint = pickle.load(savePoint_file)

pop3 = savePoint["population"]
start_gen3 = savePoint["generation"]
logbook8 = savePoint["logbook"]
random.setstate(savePoint["rndstate"])

with open("Snake9.pkl", "rb") as savePoint_file:
    savePoint = pickle.load(savePoint_file)

pop4 = savePoint["population"]
start_gen4 = savePoint["generation"]
logbook9 = savePoint["logbook"]
random.setstate(savePoint["rndstate"])



with open("logbookI005.pkl", "rb") as savePoint_file:
    savePoint = pickle.load(savePoint_file)

logbookI005pop = savePoint["population"]
logbookI005start_gen = savePoint["generation"]
logbookI005logbook = savePoint["logbook"]
random.setstate(savePoint["rndstate"])

with open("logbookI005_1.pkl", "rb") as savePoint_file:
    savePoint = pickle.load(savePoint_file)

logbookI005_1pop = savePoint["population"]
logbookI005_1start_gen = savePoint["generation"]
logbookI005_1logbook = savePoint["logbook"]
random.setstate(savePoint["rndstate"])

with open("logbookI0025.pkl", "rb") as savePoint_file:
    savePoint = pickle.load(savePoint_file)

logbookI0025pop = savePoint["population"]
logbookI0025start_gen = savePoint["generation"]
logbookI0025logbook = savePoint["logbook"]
random.setstate(savePoint["rndstate"])

with open("logbookI0025_1.pkl", "rb") as savePoint_file:
    savePoint = pickle.load(savePoint_file)

logbookI0025_1pop = savePoint["population"]
logbookI0025_1start_gen = savePoint["generation"]
logbookI0025_1logbook = savePoint["logbook"]
random.setstate(savePoint["rndstate"])

with open("logbookI25.pkl", "rb") as savePoint_file:
    savePoint = pickle.load(savePoint_file)

logbookI25pop = savePoint["population"]
logbookI25start_gen = savePoint["generation"]
logbookI25logbook = savePoint["logbook"]
random.setstate(savePoint["rndstate"])

with open("logbookI25_1.pkl", "rb") as savePoint_file:
    savePoint = pickle.load(savePoint_file)

logbookI25_1pop = savePoint["population"]
logbookI25_1start_gen = savePoint["generation"]
logbookI25_1logbook = savePoint["logbook"]
random.setstate(savePoint["rndstate"])

with open("logbookI1.pkl", "rb") as savePoint_file:
    savePoint = pickle.load(savePoint_file)

logbookI1pop = savePoint["population"]
logbookI1start_gen = savePoint["generation"]
logbookI1logbook = savePoint["logbook"]
random.setstate(savePoint["rndstate"])

with open("logbookI1_1.pkl", "rb") as savePoint_file:
    savePoint = pickle.load(savePoint_file)

logbookI1_1pop = savePoint["population"]
logbookI1_1start_gen = savePoint["generation"]
logbookI1_1logbook = savePoint["logbook"]
random.setstate(savePoint["rndstate"])

In [ ]:
%matplotlib inline
gen = logbook.select("gen")
avgData = np.array([logbook.select("avg"),
                    logbook1.select("avg"),
                    logbook2.select("avg"),
                    logbook3.select("avg"),
                    logbook4.select("avg")])

stdData = np.array([logbook.select("std"),
                    logbook1.select("std"),
                    logbook2.select("std"),
                    logbook3.select("std"),
                    logbook4.select("std")])

maxData = np.array([logbook.select("max"),
                    logbook1.select("max"),
                    logbook2.select("max"),
                    logbook3.select("max"),
                    logbook4.select("max")])

minData = np.array([logbook.select("min"),
                    logbook1.select("min"),
                    logbook2.select("min"),
                    logbook3.select("min"),
                    logbook4.select("min")])

avgs1 = np.average(avgData,axis=0)
std1 = np.average(stdData,axis=0)
max1 = np.average(maxData,axis=0)                
min1 = np.average(minData,axis=0)          

avgData = np.array([logbook5.select("avg"),
                    logbook6.select("avg"),
                    logbook7.select("avg"),
                    logbook8.select("avg"),
                    logbook9.select("avg")])

stdData = np.array([logbook5.select("std"),
                    logbook6select("std"),
                    logbook7.select("std"),
                    logbook8.select("std"),
                    logbook9.select("std")])

maxData = np.array([logbook5.select("max"),
                    logbook6select("max"),
                    logbook7.select("max"),
                    logbook8.select("max"),
                    logbook9.select("max")])

minData = np.array([logbook5.select("min"),
                    logbook6select("min"),
                    logbook7.select("min"),
                    logbook8.select("min"),
                    logbook9.select("min")])

avgs2 = np.average(avgData,axis=0)
std2 = np.average(stdData,axis=0)
max2 = np.average(maxData,axis=0)                
min2 = np.average(minData,axis=0)   


plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14) 
plt.rc('legend', fontsize=14)

fig = plt.figure(figsize=(10,7))
ax = fig.add_axes([0,0,1,1])
ax.set_xlabel("Generation")
ax.set_ylabel("Mean Fitness")
ax.plot(gen, datasAvg, lw=3, label='Final Algorithm', color='pink')
ax.fill_between(gen, avgs1-std1, avgs1 - std1, facecolor='pink', alpha=0.5)
ax.plot(gen, datasAvg1, lw=3, label='AlgYoung', color='blue')
ax.fill_between(gen, avgs2+std2, avgs2-std2, facecolor='blue', alpha=0.5)
ax.legend(loc='best', fancybox=True, framealpha=0.5)
                

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
gen = logbook1.select("gen")

indpd005= np.array([logbookI005logbook.select("avg"),logbookI005_1logbook.select("avg")])
indpd0025=np.array([logbookI0025logbook.select("avg"),logbookI0025_1logbook.select("avg")])
indpd1=np.array([logbookI1logbook.select("avg"),logbookI1_1logbook.select("avg")])
indpd25 = np.array([logbookI25logbook.select("avg"),logbookI25_1logbook.select("avg")])

avgs005=np.average(indpd005,axis=0)
avgs0025=np.average(indpd0025,axis=0)
avgs1=np.average(indpd1,axis=0)
avgs25=np.average(indpd25,axis=0)

plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14) 
plt.rc('legend', fontsize=14)

fig, ax1 = plt.subplots()
ax1.set_xlabel("Generation")
ax1.set_ylabel("Mean Fitness")

line1 = ax1.plot(gen, avgs005)
line2 = ax1.plot(gen, avgs0025)
line3 = ax1.plot(gen, avgs1)
line4 = ax1.plot(gen, avgs25)

ax1.plot(gen,avgs2, lw=3, label="indpd 0.05", color="blue")
ax1.plot(gen,avgs5, lw=3, label="indpd 0.025", color="orange")
ax1.plot(gen,avgs10, lw=3, label="indpd 0.1", color="red")
ax1.plot(gen,avgs25, lw=3, label="indpd 0.25", color="black")
ax1.legend(loc="best", fancybox=True,framealpha=0.5)
